In [1]:
import asyncio
import importlib
import os
import warnings

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm  # For multi-model support
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.tool_context import ToolContext
from google.genai import types  # For creating message Content/Parts
from IPython.display import HTML, Markdown, display

# Ignore all warnings
warnings.filterwarnings("ignore")

import logging

logging.basicConfig(level=logging.ERROR)

# Set `PATH` to include the directory containing ADK command
# PATH = %env PATH
# %env PATH=/home/jupyter/.local/bin:{PATH}

In [2]:
LOCATION = "us-central1"
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"  # Use Vertex AI API
os.environ["GOOGLE_CLOUD_PROJECT"] = "qwiklabs-asl-01-81978728819b"  # Use Vertex AI API

In [3]:
%%bash
# -p オプションを付けて adk_agents/ ディレクトリを作成します。
# -p は、ディレクトリが既に存在してもエラーを出さず、親ディレクトリも同時に作成します。
mkdir -p adk_agents

In [4]:
%%bash
echo > adk_agents/.env "GOOGLE_CLOUD_LOCATION=$GOOGLE_CLOUD_LOCATION
GOOGLE_CLOUD_PROJECT=$GOOGLE_CLOUD_PROJECT

GOOGLE_GENAI_USE_VERTEXAI=$GOOGLE_GENAI_USE_VERTEXAI
"

In [5]:
MODEL = "gemini-2.0-flash"

In [6]:
%%bash
# 必要なディレクトリ構造をすべて作成します。
# -p オプションは、パスの中間にあるディレクトリもすべて作成し、
# すでに存在してもエラーを出さないため安全です。
mkdir -p ./adk_agents/agent1_news_collector

In [27]:
%%writefile ./adk_agents/agent1_news_collector/tools.py

import pandas as pd
from GoogleNews import GoogleNews # GoogleNewsなどの必要なライブラリもインポート
import json
from google import genai
from google.genai.errors import APIError

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


def fetch_stock_news_from_google_news(query):
    """Retrieves recent news articles related to a specific financial or market query from Google News.

    Args:
        query (str): The keyword or phrase for the search (e.g., "Toyota stock forecast", "Nikkei 225 news").
        
    Returns:
        dict: A Data dictionary containing the collected news data (e.g., Title, Summary, Source, URL).
    """
    print(f"✅ 検索キーワード: '{query}' でGoogle Newsを検索中...")

    # GoogleNewsオブジェクトを初期化
    googlenews = GoogleNews(lang='en', region='US') # 日本語、日本リージョンで設定
    # googlenews = GoogleNews(lang='ja', region='ja') # 日本語、日本リージョンで設定
    # 検索クエリを設定し、指定ページ数まで結果を取得
    googlenews.search(query)
    
    # 結果を取得
    results = googlenews.results()
    
    news_data = {} # 辞書型として初期化
    article_index = 1 # 記事のキーとして使用するカウンター

    for item in results:
        # 必要な情報を抽出し、一時的なディクショナリ（記事データ）を作成
        article_dict = {
            'title': item.get('title'),
            'source': item.get('publisher'),
            'date': item.get('date'),
            'url': item.get('link'),
            'summary': item.get('desc')
        }
    
        # 💡 修正点: .append() の代わりに、連番をキーとしてディクショナリに保存
        news_data[f'article_{article_index}'] = article_dict
        
        # 次のキーのためにカウンターを増やす
        article_index += 1
    
    if not news_data:
        print("❌ 該当するニュース記事が見つかりませんでした。")
        return {
            "status": "failed",
            "count": 0,
            "message": "No news articles found for the query.",
            "articles": []
        }

    print(f"✅ 合計 {len(news_data)} 件の記事が見つかりました。")
    print("#######################",news_data)
    # ファイルに書き込む
    with open("./data.json", 'w', encoding='utf-8') as f:
        # indent=4 で整形して書き込むと、ファイルが読みやすくなります
        json.dump(news_data, f)

    return {"status": "OK"}


def predict_index(target_index: str, query: str) -> dict:
    """The minimum function required to analyze news articles and have Gemini generate a prediction and its rationale."""
    # ----------------------------------------------------
    # 💡 注意: clientは通常モジュールレベルで一度だけ初期化します
    # ファイルを読み込む
    with open("./data.json", 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print("######data",data)
    try:
        client = genai.Client()
        print("Gemini Clientの初期化しました")
    except Exception as e:
        # 環境変数または認証情報に問題がある可能性があります
        print(f"Gemini Clientの初期化に失敗しました: {e}")
        client = None
    # ----------------------------------------------------
    # 1. 最小限の初期チェック
    if client is None:
        return {"status": "error", "message": "Client not initialized."}

    if not query:
        return {"status": "data_insufficient", "message": "No news data provided."}

    
    # 3. 最小限のプロンプトと必須JSON形式
    prompt = f"""
    Based *only* on the provided news articles in the {data}, analyze the sentiment and predict the closing direction for the **{target_index}** index. 
    Output the result strictly in the required JSON format.

    News Articles:
    ---
    {query}
    ---

    Required JSON Format:
    {{
      "predicted_close": <float or current index value>,
      "market_sentiment": "<Bullish|Bearish|Neutral>",
      "analysis_basis": "<Concise summary of the market drivers, max 50 words>"
    }}
    """
    
    # 4. API呼び出しと最小限のエラー処理
    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=prompt,
            config={"response_mime_type": "application/json"}
        )
        
        # JSON応答をパース
        llm_analysis = json.loads(response.text)
        
        # 💡 デバッグ用: 応答が成功した場合、Geminiの解析結果をそのまま返して構造を確認
        return {
            "prediction_status": "success",
            "target_index": target_index,
            "llm_output": llm_analysis # 解析結果全体を返して確認
        }

    except Exception as e:
        # ネットワーク、認証、JSONパースなど、全てのエラーをキャッチ
        return {
            "prediction_status": "runtime_error",
            "message": f"An unexpected error occurred during API call or JSON parsing: {type(e).__name__}: {str(e)}",
            "error_type": type(e).__name__
        }
    

Overwriting ./adk_agents/agent1_news_collector/tools.py


In [16]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
%autoreload 2

In [18]:
%%writefile ./adk_agents/agent1_news_collector/agent.py
from google.adk.agents import Agent
from google.adk.tools import AgentTool

from .tools import fetch_stock_news_from_google_news, predict_index

google_search_agent = Agent(
    name="google_search_agent",
    model="gemini-2.0-flash",
    instruction="Retrieves recent news articles from Google News.",
    tools=[fetch_stock_news_from_google_news]
)

predict_agent = Agent(
    name="predict_index_price_agent",
    model="gemini-2.0-flash",
    instruction="Predict the closing price of a stock index based on news articles.",
    tools=[predict_index]
)

root_agent = Agent(
    name="news_agent_v1",
    model="gemini-2.0-flash", # Can be a string for Gemini or a LiteLlm object
    description="Analyzes market news to predict stock index trends.",
    instruction="You are a helpful finance assistant. When the user asks for a prediction about a specific index(like DOW or N225), use the 'predict_index_price_agent' tool to provide a prediction.",

    tools=[
        
        AgentTool(google_search_agent, skip_summarization=False),
        AgentTool(predict_agent, skip_summarization=False)
        
    ]
)

Overwriting ./adk_agents/agent1_news_collector/agent.py


In [28]:
from adk_agents.agent1_news_collector import agent
from adk_agents.agent1_news_collector import tools

importlib.reload(agent)  # Force reload
importlib.reload(tools)  # Force reload

# Example tool usage (optional test)
print(agent.fetch_stock_news_from_google_news("Dow jones"))


print(agent.predict_index("n225","US stock market today: Nasdaq dropped 1.9%, S&P 500 lost 1.1%, Dow slipped 0.8% — What led to Wall Street turning red and why Tesla, Nvidia, Amazon, and Palantir sank"))


✅ 検索キーワード: 'Dow jones' でGoogle Newsを検索中...
✅ 合計 10 件の記事が見つかりました。
####################### {'article_1': {'title': 'Dow Jones & Nasdaq 100: China Trade Slump Hits US Futures in Asia', 'source': None, 'date': '2 hours ago', 'url': 'https://www.fxempire.com/forecasts/article/dow-jones-nasdaq-100-china-trade-slump-hits-us-futures-in-asia-1559783&ved=2ahUKEwiWiMX9vt-QAxVH4MkDHd30IRQQxfQBegQIBBAC&usg=AOvVaw0msDAMf-3R0a2iZkQ13tCK', 'summary': 'Asian markets slipped after Chinese exports plunged and US-China trade risks resurfaced, keeping Dow Jones and Nasdaq 100 futures under pressure ahead of...'}, 'article_2': {'title': 'Breakingviews - Stock market frenzy is a louder echo of the 1990s', 'source': None, 'date': '2 hours ago', 'url': 'https://www.reuters.com/commentary/breakingviews/stock-market-frenzy-is-louder-echo-1990s-2025-11-07/&ved=2ahUKEwiWiMX9vt-QAxVH4MkDHd30IRQQxfQBegQICRAC&usg=AOvVaw3Tty2t1xByt1dsS0iaTQRh', 'summary': 'The tech analyst Fred Hickey developed a cult following during

In [20]:
APP_NAME = "stocks_market_app"
USER_ID = "user_1"
SESSION_ID = "session_010"  # Using a fixed ID for simplicity

In [21]:
session_service = InMemorySessionService()

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)

In [22]:
runner = Runner(
    agent=agent.root_agent,  # The agent we want to run
    app_name=APP_NAME,  # Associates runs with our app
    session_service=session_service,  # Uses our session manager
)

In [29]:
async def call_agent_async(query: str, runner, user_id, session_id):
    """Sends a query to the agent and prints the final response."""
    print(f"\n>>> User Query: {query}")

    content = types.Content(role="user", parts=[types.Part(text=query)])

    final_response_text = "Agent did not produce a final response."  # Default

    # Key Concept: run_async executes the agent logic and yields Events.
    # We iterate through events to find the final answer.
    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=content
    ):
        # You can uncomment the line below to see *all* events during execution
        # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

        # Key Concept: is_final_response() marks the concluding message for the turn.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Assuming text response in the first part
                final_response_text = event.content.parts[0].text
            elif (
                event.actions and event.actions.escalate
            ):  # Handle potential errors/escalations
                final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
            # Add more checks here if needed (e.g., specific error codes)
            break  # Stop processing events once the final response is found

    print(f"<<< Agent Response: {final_response_text}")

In [30]:
await call_agent_async(
    "search November 7th 2025 news,and then  predict today's DOW Jones-close-price",
    runner=runner,
    user_id=USER_ID,
    session_id=SESSION_ID,
)


>>> User Query: search November 7th 2025 news,and then  predict today's DOW Jones-close-price


ValueError: Failed to parse the parameter query of function fetch_stock_news_from_google_news for automatic function calling. Automatic function calling works best with simpler function signature schema, consider manually parsing your function declaration for function fetch_stock_news_from_google_news.

In [ ]:
# !adk web adk_agents